In [1]:
!git clone https://github.com/CCogS-Mx/text-preprocessing.git

fatal: destination path 'text-preprocessing' already exists and is not an empty directory.


## Imports

In [2]:
import sys
sys.path.append('text-preprocessing/script/')
sys.path.append('Scripts/')

In [3]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Import classes
import text_preprocessing as tp
import feature_extraction as fe
import robust_classic_model as rcm
import cleaning_twitter_data as ctd

# Algoritmos de ML
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV

# Vectorizadores
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\qk_le\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\qk_le\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\qk_le\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Data

In [4]:
main_path = 'Data/'
training = pd.read_csv(f'{main_path}homomex_training.csv').drop('index', axis=1)

In [5]:
training.head()

,tweets,label
0,"Me quise ligar a una chava ayer y no me pelo, ...",P
1,"@papaya_rockera eres un puñal, Papayita.",P
2,Magnate ofrece 130 mdd al hombre que conquiste...,P
3,Los trolebuses del desgobierno de @EPN son idi...,P
4,"En época de Hitler no se decía ""eres gay"" y, s...",P


In [6]:
prep = tp.Preprocessing('spanish')

In [7]:
training_p = prep.main_preprocess(data=training, 
                                  column='tweets', 
                                  tweet=True, 
                                  tweet_tags=True, 
                                  remove_stop_words=False, 
                                  lemmatize=False, 
                                  translate_emojis=True)

In [8]:
training_p.head()

,tweets,label
0,me quise ligar a una chava ayer y no me pelo l...,P
1,eres un punal papayita,P
2,magnate ofrece mdd al hombre que conquiste a s...,P
3,los trolebuses del desgobierno de son idiotas ...,P
4,en epoca de hitler no se decia eres gay y si e...,P


In [9]:
training_p.label = training_p.label.fillna('NR')

In [10]:
dictionary_list = [{'P': 0, 'NP': 1, 'NR': 2}]

In [11]:
training_p.label.replace(list(dictionary_list[0].keys()), 
            list(dictionary_list[0].values()), 
            inplace=True)

In [12]:
training_p.label.value_counts()

label
1    4360
2    1778
0     862
Name: count, dtype: int64

## Finding best hyperparams

In [13]:
models = [LogisticRegression(), RandomForestClassifier(), LinearSVC(), KNeighborsClassifier()]
model_names = ['Regresión logística', 'Random Forest', 'SVM Lineal', 'KNN']
x_label_column = 'tweets'
y_label_column = 'label'
folds = 5
grid_params  = [{"C":np.logspace(-3,3,7), "penalty":["l2"], "max_iter": [1000, 10000], "random_state":[42]},
                {"n_estimators": [10, 50, 100], "max_depth":[10, 50, 100], "random_state": [42]},
                {"C":np.logspace(-3,3,7), "penalty":["l2"], "max_iter": [1000, 10000], "random_state":[42]},
                {"n_neighbors":[3], "weights":['uniform', 'distance']}]
path_to_save_models = ''
path_to_save_vectorizers = ''

In [14]:
for i in range(2):
  if i == 0:
    print('Count vectorizer')
    vectorizer = CountVectorizer(ngram_range = (1,1), lowercase = False)
    oth_feats_vectorizer = CountVectorizer(ngram_range = (1,1), lowercase = False)
  else:
    print('TF-IDF')
    vectorizer = TfidfVectorizer(ngram_range = (1,1), lowercase = False)
    oth_feats_vectorizer = TfidfVectorizer(ngram_range = (1,1), lowercase = False)

  ml_homo = rcm.RobustMachineLearningModel(models=models,
                                       model_names=model_names,
                                       train_data=training_p,
                                       x_label_column=x_label_column,
                                       y_label_column=y_label_column,
                                       folds=folds,
                                       grid_params=grid_params,
                                       path_to_save_models=path_to_save_models,
                                       path_to_save_vectorizer=path_to_save_vectorizers,
                                       lemma=True,
                                       pos=True,
                                       tag=True,
                                       other_features=True,
                                       vectorizer=vectorizer,
                                       oth_feats_vectorizer=oth_feats_vectorizer,
                                       language='es')
  ml_homo.get_best_cross_validation_pca()

Count vectorizer
Model 0: Regresión logística
Fitting 5 folds for each of 14 candidates, totalling 70 fits
[CV 1/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.342, test=0.327) total time=   0.5s
[CV 2/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.351, test=0.340) total time=   0.5s
[CV 3/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.335, test=0.314) total time=   0.5s
[CV 4/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.341, test=0.350) total time=   0.5s
[CV 5/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.330, test=0.325) total time=   0.4s
[CV 1/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.342, test=0.327) total time=   0.5s
[CV 2/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.351, test=0.340) total time=   0.5s
[CV 3/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42

In [15]:
for i in range(2):
  if i == 0:
    print('Count vectorizer')
    vectorizer = CountVectorizer(ngram_range = (1,2), lowercase = False)
    oth_feats_vectorizer = CountVectorizer(ngram_range = (1,2), lowercase = False)
  else:
    print('TF-IDF')
    vectorizer = TfidfVectorizer(ngram_range = (1,2), lowercase = False)
    oth_feats_vectorizer = TfidfVectorizer(ngram_range = (1,2), lowercase = False)

  ml_homo = rcm.RobustMachineLearningModel(models=models,
                                       model_names=model_names,
                                       train_data=training_p,
                                       x_label_column=x_label_column,
                                       y_label_column=y_label_column,
                                       folds=folds,
                                       grid_params=grid_params,
                                       path_to_save_models=path_to_save_models,
                                       path_to_save_vectorizer=path_to_save_vectorizers,
                                       lemma=True,
                                       pos=True,
                                       tag=True,
                                       other_features=True,
                                       vectorizer=vectorizer,
                                       oth_feats_vectorizer=oth_feats_vectorizer,
                                       language='es')
  ml_homo.get_best_cross_validation_pca()

Count vectorizer
Model 0: Regresión logística
Fitting 5 folds for each of 14 candidates, totalling 70 fits
[CV 1/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.400, test=0.381) total time=   0.6s
[CV 2/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.403, test=0.381) total time=   0.6s
[CV 3/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.394, test=0.352) total time=   0.5s
[CV 4/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.399, test=0.381) total time=   0.5s
[CV 5/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.378, test=0.379) total time=   0.5s
[CV 1/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.400, test=0.381) total time=   0.5s
[CV 2/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.403, test=0.381) total time=   0.5s
[CV 3/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42

In [16]:
for i in range(2):
  if i == 0:
    print('Count vectorizer')
    vectorizer = CountVectorizer(ngram_range = (1,3), lowercase = False)
    oth_feats_vectorizer = CountVectorizer(ngram_range = (1,3), lowercase = False)
  else:
    print('TF-IDF')
    vectorizer = TfidfVectorizer(ngram_range = (1,3), lowercase = False)
    oth_feats_vectorizer = TfidfVectorizer(ngram_range = (1,3), lowercase = False)

  ml_homo = rcm.RobustMachineLearningModel(models=models,
                                       model_names=model_names,
                                       train_data=training_p,
                                       x_label_column=x_label_column,
                                       y_label_column=y_label_column,
                                       folds=folds,
                                       grid_params=grid_params,
                                       path_to_save_models=path_to_save_models,
                                       path_to_save_vectorizer=path_to_save_vectorizers,
                                       lemma=True,
                                       pos=True,
                                       tag=True,
                                       other_features=True,
                                       vectorizer=vectorizer,
                                       oth_feats_vectorizer=oth_feats_vectorizer,
                                       language='es')
  ml_homo.get_best_cross_validation_pca()

Count vectorizer
Model 0: Regresión logística
Fitting 5 folds for each of 14 candidates, totalling 70 fits
[CV 1/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.424, test=0.390) total time=   0.8s
[CV 2/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.429, test=0.391) total time=   0.7s
[CV 3/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.421, test=0.368) total time=   1.2s
[CV 4/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.420, test=0.393) total time=   0.7s
[CV 5/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.400, test=0.400) total time=   0.7s
[CV 1/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.424, test=0.390) total time=   0.7s
[CV 2/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.429, test=0.391) total time=   0.6s
[CV 3/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42